### JS's Promise.resolve equivalent

In JS, you can wrap a value in a dummy promise, if the value is itself a promise, returns itself

In [9]:
asyncio.run(resolve(10))

10

In [10]:
import asyncio
from collections.abc import Awaitable


async def resolve(coro):
    if isinstance(coro, Awaitable):
        return await coro
    return coro

In [11]:
asyncio.run(resolve(10))

10

In [12]:
async def foo():
    return 42

asyncio.run(resolve(foo()))

42

## @sync decorator

A decorator that turns a async function into a normal function, auto calling asyncio.run

In [1]:
import asyncio
from functools import wraps


def sync(f):
    @wraps(f)
    def _sync(*args, **kwargs):
        return asyncio.run(f(*args, **kwargs))
    return _sync

In [2]:
@sync
async def foo():
    return 42

foo()

42

## @agnostic decorator

Turns a normal coroutine function into a agnostic version of it, that is, if called outside a running loop, awaits it with asyncio.run

In [11]:
import asyncio
from functools import wraps
from collections.abc import Awaitable


def agnostic(f):
    @wraps(f)
    def _agnostic(*args, **kwargs):
        result = f(*args, **kwargs)
        if not isinstance(result, Awaitable):
            return result
        try:
            asyncio.get_running_loop()
            return result
        except RuntimeError:
            return asyncio.run(result)
    return _agnostic

In [14]:
@agnostic
async def foo():
    return 42

foo()

42

In [15]:
async def main():
    return await foo()

asyncio.run(main())

42

In [16]:
@agnostic
def sync_foo():
    return 42

sync_foo()

42

## AsyncQueue

A queue-like with a `submit` and a `get_done` method for coroutine running

In [1]:
import asyncio


class AsyncQueue:
    def __init__(self, *args, loop=None, **kwargs):
        if loop is None:
            self._loop = asyncio.get_event_loop()
        else:
            self._loop = loop
        self._done = asyncio.Queue(*args, loop=loop, **kwargs)

    def submit(self, f, *args, **kwargs):
        async def _():
            result = await f(*args, **kwargs)
            await self._done.put(result)
            return result
        return self._loop.create_task(_())

    def get_done(self):
        return self._done.get()

In [2]:
async def wrap(x):
    return x


q = AsyncQueue()

for i in range(5):
    q.submit(wrap, i)


async def main():
    for i in range(5):
        print(await q.get_done())


asyncio.get_event_loop().run_until_complete(main())

0
1
2
3
4


## as_completed_map

Maps a coroutine function to a iterable, yielding completed results

In [3]:
import asyncio


async def as_completed_map(f, it):
    _ = map(f, it)
    _ = map(asyncio.ensure_future, _)
    _ = asyncio.as_completed(list(_))

    for completed in _:
        yield await completed

In [5]:
async def sleep_print(x):
    await asyncio.sleep(x)
    return x


async def main():
    nums = [4, 3, 5, 1, 2]
    async for el in as_completed_map(sleep_print, nums):
        print(el)

asyncio.run(main())

1
2
3
4
5
